In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer

from hashlib import sha256
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf

pd.set_option('display.max_columns', None)

output = join("output","correcoes","tb_pacientes")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer_all = Timer()
timer_all.reset()

In [3]:
timer.reset()
tb = TbPacientes()
tb.update()
timer.stop()

125.2616195678711

In [4]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()

Casos confirmados excluidos: 0
Obitos confirmados excluidos: 0


192.8892707824707

In [5]:
casost = tb.get()
casosc = casos_confirmados.get_casos()
print(f"casos casos_confirmados: {casosc.shape}")
print(f"casos tb_pacientes: {casost.shape}")

casos casos_confirmados: (430873, 21)
casos tb_pacientes: (430873, 48)


In [6]:
idx_casos_confirmados = casosc[casosc['hash'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_tb_pacientes = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')

In [7]:
idx_tb_pacientes = casost[casost['hash'].isin(casosc['hash'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_less'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_more'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_less_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_more_atend'])].index.tolist()
idx_tb_pacientes = set(idx_tb_pacientes)
tb_pacientes_nao_casos_confirmados = casost.loc[ set(casost.index.tolist()) - idx_tb_pacientes ].sort_values('identificacao')


In [8]:
print(f"casos_confirmados_nao_tb_pacientes: {casos_confirmados_nao_tb_pacientes.shape}")
if len(casos_confirmados_nao_tb_pacientes) > 0:
    casos_confirmados_nao_tb_pacientes.sort_values('nome').to_excel(join(output,'casos_confirmados_nao_tb_pacientes.xlsx'), index=False)
print(f"tb_pacientes_nao_casos_confirmados: {tb_pacientes_nao_casos_confirmados.shape}")
if len(tb_pacientes_nao_casos_confirmados) > 0:
    tb_pacientes_nao_casos_confirmados.sort_values('nome').to_excel(join(output,'tb_pacientes_nao_casos_confirmados.xlsx'), index=False)

casos_confirmados_nao_tb_pacientes: (0, 21)
tb_pacientes_nao_casos_confirmados: (0, 48)


In [9]:
casost['ibge_res_pr'] = casost['ibge_res_pr'].fillna('9999999')
casosc['ibge7'] = casosc['ibge7'].fillna('9999999')

In [10]:
casosc['nome_hash'] = casosc['nome'].apply(normalize_hash)
casost['nome_hash'] = casost['nome'].apply(normalize_hash)

In [11]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
# exit()

True
True


SystemExit: 

In [12]:
timer.reset()

sorted_casosc = pd.DataFrame(columns=casosc.columns, dtype=casosc.dtypes.values)
for mun, df_mun in casosc.groupby('ibge7'):
    sorted_casosc = sorted_casosc.append(df_mun.sort_values('nome_hash'))

sorted_casost = pd.DataFrame(columns=casost.columns, dtype=casost.dtypes.values)
for mun, df_mun in casost.groupby('ibge_res_pr'):
    sorted_casost = sorted_casost.append(df_mun.sort_values('nome_hash'))

casosc = sorted_casosc
print(f"casos casos_confirmados: {sorted_casosc.shape}")
casost = sorted_casost
print(f"casos tb_pacientes: {casost.shape}")
print(timer.stop())

casos casos_confirmados: (430873, 22)
casos tb_pacientes: (430873, 49)
482.1813042163849


In [13]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))

True
True


In [14]:
casosc = casosc.drop(columns=['hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend'])
casost = casost.drop(columns=['hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend'])

In [15]:
casosc['idade_mais1'] = casosc['idade'] + 1
casosc['idade_menos1'] = casosc['idade'] - 1

In [16]:
writer = pd.ExcelWriter(join(output,"novos_tb_pacientes.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casost.to_excel(writer,index=None)

writer.save()
writer.close()

In [17]:
writer = pd.ExcelWriter(join(output,"casos_ordenados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casosc.to_excel(writer,index=None)

writer.save()
writer.close()

In [18]:
print(timedelta(seconds=timer_all.stop()))
exit()

0:20:30.798044


SystemExit: 

In [6]:
# casost = tb.get()
# casost['nome_hash'] = casost['nome'].apply(normalize_hash)
writer = pd.ExcelWriter("obitos_tb.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')


obitos_tb = pd.DataFrame(columns=casost.columns, dtype=casost.dtypes.values)

for dt, df in casost.loc[casost['obito']=='SIM'].sort_values('dt_obito').groupby('dt_obito'):
    obitos_tb = obitos_tb.append(df.sort_values('nome_hash'))

obitos_tb[['nome','sexo','idade','mun_resid','telefone','dt_obito']].to_excel(writer,index=None)
writer.save()
writer.close()

In [7]:
obitos_tb

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,15_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais1,idade_menos1,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,nome_hash
293481,27696,4115200,4126256,ROSANGELA ANTUNES MACHADO DA,F,54,MARINGA,SARANDI,LACEN,2020-03-27,2020-03-27,2020-03-14,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2020-03-25,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,54,NaT,NaT,55,53,59ddac8129df2d9e339ecc671d36e53a53f812cba174e7...,9827fb8afb6f465e768d7f0a09e52e62355a3e4a54d6b8...,5b0b700a486c9659c49d856e09ee02eb2ff297cc26ac79...,e8488ec6367efb1127b722b5c09a68ed4e90d03f7ec172...,6d82d6f8db00afc3869d928155eb1035174b2755465280...,fa47ef968b5d880fc8530fb7475370fb92afb295c5b581...,ROSANGELAANTUNESMACHADODA
276763,4062,4115200,4115200,BELCHIOR MARTINS,M,84,MARINGA,MARINGA,LACEN,2020-03-27,2020-03-27,2020-03-15,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2020-03-26,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,84,NaT,NaT,85,83,85ac0220b22d0ab62805b5f67d04bb9a462261ac5bfb1d...,ddefa6d9385b2c7157f6adb012cc70d6004bf0c74179f5...,d4abd6637cc2194cd01e3639567592309bb2043f918071...,85ac0220b22d0ab62805b5f67d04bb9a462261ac5bfb1d...,ddefa6d9385b2c7157f6adb012cc70d6004bf0c74179f5...,d4abd6637cc2194cd01e3639567592309bb2043f918071...,BELCHIORMARTINS
70183,24247,4104808,4104808,NEI SEUTER MARTINS,M,66,CASCAVEL,CASCAVEL,LACEN,2020-03-29,2020-03-29,2020-03-14,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2020-03-30,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,66,NaT,NaT,67,65,c9e6c2af63c81a31de8c998fc6f91febb96021499aa4ba...,5ffbd5c6a02f08c47c80d877e0767bc10682fbbbba8bca...,120921da96f334c132444ecaaa5314e2c03bba78201054...,c9e6c2af63c81a31de8c998fc6f91febb96021499aa4ba...,5ffbd5c6a02f08c47c80d877e0767bc10682fbbbba8bca...,120921da96f334c132444ecaaa5314e2c03bba78201054...,NEISEUTERMARTINS
52878,11387,4104303,4104303,GABRIEL GOLIN,M,72,CAMPO MOURAO,CAMPO MOURAO,LACEN,2020-04-02,2020-04-02,2020-03-18,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2020-03-31,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,72,NaT,NaT,73,71,e1e0944bb26f111441f1971caf03e742544eeb2d50ac4f...,61728a5540431f80de24ea030516462aa6434b7f8500ca...,e7ae21cfb4abde4364af06d95d63741cc85e2ddaf01c88...,e1e0944bb26f111441f1971caf03e742544eeb2d50ac4f...,61728a5540431f80de24ea030516462aa6434b7f8500ca...,e7ae21cfb4abde4364af06d95d63741cc85e2ddaf01c88...,GABRIELGOLIN
112941,179699,4106902,4106902,DENISE STANKEVICZ CORREA,F,40,CURITIBA,CURITIBA,HSNG,2020-04-03,2020-08-27,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2020-04-02,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,43,40,2020-08-27,NaT,41,39,49b518777958059c93e1ff35155c60475ac9808c0c12a4...,88f05f92ea8cf7fb96ff5302ad0a32e7af5747591ff002...,820e78062fd5f1ef4610b0290ba346458ffae5ccf6685f...,49b518777958059c93e1ff35155c60475ac9808c0c12a4...,88f05f92ea8cf7fb96ff5302ad0a32e7af5747591ff002...,820e78062fd5f1ef4610b0290ba346458ffae5ccf6685f...,DENISESTANKEVICZCORREA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325227,501970,4118402,4126256,RICARDO AUDI,M,66,PARANAVAI,SARANDI,COVID-19 BIOLOGIA MOLECULAR,2020-12-21,2021-01-05,2020-11-30,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2021-01-05,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,130,66,2021-01-05,NaT,67,65,c1fb3abc07f67e039af0c4362321e06747754741a221b4...,3597032f2228f9ade74bc814912e31ba8fa3bf9783f9c6...,9a2234045bb6e1d907caaa4c40fe2262836c58ff495c34...,479d0aa59723a8cf84d4e8d2db47e58b892b8bec6249f2...,e8c357e74c97d256bc7c78c870